# Question 1: Package imports (1 point)

Please load in all software imports (e.g., `from google.colab import drive`) here.

In [26]:
# download spacy language model and restart runtime if necessary
!pip3 install spacy nltk transformers
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [27]:
from google.colab import drive, files
import json
from pprint import pprint

import spacy
import numpy as np
import os

from transformers import BertTokenizer

b_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")


# load in spacy
nlp = spacy.load("en_core_web_lg")

In [28]:
# change this to work with your setup
drive.mount("/content/drive/", force_remount=True)

with open("/content/drive/MyDrive/Fall 2021 Computational Linguistics Notebooks/HW3/morphemes.json") as data:
  file_contents = [x for x in data.readlines()]

file_contents

Mounted at /content/drive/


['{"word": "acclimatization", "roots": ["climate"], "prefixes": ["a"], "suffixes": ["ise", "ation"]}\n',
 '{"word": "acclimatize", "roots": ["climate"], "prefixes": ["a"], "suffixes": ["ise"]}\n',
 '{"word": "acclimatized", "roots": ["climate"], "prefixes": ["a"], "suffixes": ["ise", "ed"]}\n',
 '{"word": "acclimatizes", "roots": ["climate"], "prefixes": ["a"], "suffixes": ["ise", "s"]}\n',
 '{"word": "acclimatizing", "roots": ["climate"], "prefixes": ["a"], "suffixes": ["ise", "ing"]}\n',
 '{"word": "accreditation", "roots": ["credit"], "prefixes": ["a"], "suffixes": ["a", "tion"]}\n',
 '{"word": "accredited", "roots": ["credit"], "prefixes": ["a"], "suffixes": ["ed"]}\n',
 '{"word": "accrediting", "roots": ["credit"], "prefixes": ["a"], "suffixes": ["ing"]}\n',
 '{"word": "accredits", "roots": ["credit"], "prefixes": ["a"], "suffixes": ["s"]}\n',
 '{"word": "aeroplanes", "roots": ["plane"], "prefixes": ["aero"], "suffixes": ["s"]}\n',
 '{"word": "afforestation", "roots": ["forest"], 

# Question 2: Converting the morphemes.json file to a dictionary (5 points)

Now, read in each line, convert into a dictionary (using the `.loads` function from the `json` package), and append to a `list` called `morphemes` whose elements are each `dict` files.

You may load in the data any way you like that results in turning each line of `morphemes.json` into a dictionary. To do this, you can use the `json.loads` function.

```python
one_line_dict = json.loads('{"word": '', "prefixes": [], "suffixes": [], "roots": []}')
word_of_one_line = one_line_dict['word']
new_one_line_dict = {word_of_one_line: one_line_dict}
```

Combine each of the dictionaries from each line into a single dictionary (keys are `word`, values are the whole dictionary):

```python
my_dict = {
  'word1': {'word': 'word1',
            'roots': ['word'],
            'prefixes'=[],
            'suffixes': ['1']}
}
```

In [29]:
# now convert to the morphemes data structure described above
morphemes = []
for i, line in enumerate(file_contents):
  morphemes.append(json.loads(line))

print(morphemes)

morphemes_dict = {}
for i, morpheme_dict in enumerate(morphemes):
  word = morpheme_dict['word']
  morphemes_dict[word] = morpheme_dict

print(morphemes_dict)

[{'word': 'acclimatization', 'roots': ['climate'], 'prefixes': ['a'], 'suffixes': ['ise', 'ation']}, {'word': 'acclimatize', 'roots': ['climate'], 'prefixes': ['a'], 'suffixes': ['ise']}, {'word': 'acclimatized', 'roots': ['climate'], 'prefixes': ['a'], 'suffixes': ['ise', 'ed']}, {'word': 'acclimatizes', 'roots': ['climate'], 'prefixes': ['a'], 'suffixes': ['ise', 's']}, {'word': 'acclimatizing', 'roots': ['climate'], 'prefixes': ['a'], 'suffixes': ['ise', 'ing']}, {'word': 'accreditation', 'roots': ['credit'], 'prefixes': ['a'], 'suffixes': ['a', 'tion']}, {'word': 'accredited', 'roots': ['credit'], 'prefixes': ['a'], 'suffixes': ['ed']}, {'word': 'accrediting', 'roots': ['credit'], 'prefixes': ['a'], 'suffixes': ['ing']}, {'word': 'accredits', 'roots': ['credit'], 'prefixes': ['a'], 'suffixes': ['s']}, {'word': 'aeroplanes', 'roots': ['plane'], 'prefixes': ['aero'], 'suffixes': ['s']}, {'word': 'afforestation', 'roots': ['forest'], 'prefixes': ['a'], 'suffixes': ['ation']}, {'word':

# Question 3: Identify roots, suffixes, and prefixes for one word (2 points)

Identify the root, suffixes, and prefixes of the word `"transformations"`

In [30]:
print(morphemes_dict['transformations'])

{'word': 'transformations', 'roots': ['form'], 'prefixes': ['trans'], 'suffixes': ['ation', 's']}


# Question 4: Make dictionaries mapping between morphemes and words (5 points)

* Roots --> Set of all words containing that root
* Prefixes --> Set of all words containing that prefix
* Suffixes --> Set of all words containing that suffix

**Note**: Be sure to loop over all words, and within each morpheme category, be sure to loop over all morphemes.

Tip: If you use a `defaultdict`, this might make some of the work easier. Here is a `defaultdict` example based on the n-gram frequency calculations from previous lectures and the previous homework. It allows us to set default `value`s for a `key`. For this assignment, the default value is a `set`. Note, you do not _have_ to use this data structure.

```python
from collections import defaultdict

word_frequency_dict = defaultdict(int)
list_of_words = ['this', 'is', 'a', 'sentence']

for word in list_of_words:
  word_frequency_dict[word] += 1
```

In [31]:
breakdown = {'roots': {}, 'suffixes': {}, 'prefixes': {}}


for morpheme in morphemes:
    word = morpheme['word']
    for keys, values in enumerate(morpheme):
        if values == 'roots':
            for root in morpheme['roots']:
                if not root in breakdown['roots']:
                    breakdown['roots'][root] = {word}
                else:
                    breakdown['roots'][root].add(word)
        elif values == 'prefixes' and len(morpheme['prefixes']) > 0:
            for prefix in morpheme['prefixes']:
                if not prefix in breakdown['prefixes']:
                    breakdown['prefixes'][prefix] = {word}
                else:
                    breakdown['prefixes'][prefix].add(word)
        elif values == 'suffixes' and len(morpheme['suffixes']) > 0:
            for suffix in morpheme['suffixes']:
                if not suffix in breakdown['suffixes']:
                    breakdown['suffixes'][suffix] = {word}
                else:
                    breakdown['suffixes'][suffix].add(word)

print(breakdown)


{'roots': {'climate': {'acclimatizes', 'climate', 'acclimatizing', 'acclimatize', 'acclimatized', 'acclimatization'}, 'credit': {'accreditation', 'accredited', 'accrediting', 'discrediting', 'accredits', 'discredits', 'discredited', 'credit', 'discreditable', 'discreditably'}, 'plane': {'biplanes', 'plane', 'aquaplaned', 'enplanes', 'aquaplaning', 'deplanes', 'monoplanes', 'aeroplanes', 'deplaned', 'hydroplanes', 'aquaplanes', 'deplaning', 'enplaned', 'enplaning'}, 'forest': {'afforesting', 'disafforesting', 'afforestation', 'reafforests', 'disforested', 'deforesting', 'reforested', 'reforests', 'disafforested', 'deforestation', 'afforests', 'disafforests', 'disforesting', 'reafforesting', 'reforestations', 'afforested', 'reforestation', 'reafforested', 'disafforestation', 'deforested', 'reforesting', 'reafforestation', 'deforests', 'disforests', 'forest'}, 'morale': {'amorality', 'morale'}, 'aerobe': {'anaerobic', 'aerobe'}, 'ion': {'anionic', 'anions', 'ion'}, 'chamber': {'antechambe

# Question 5: Cosine similarity with spaCy (8 points)

Compute the mean cosine similarity using `numpy.mean` for the word `"transformations"` to all other words that share:

1. Same root
2. Same suffixes
3. Same prefixes



In [38]:
cos_similarity_list = []
for i, word in enumerate(morphemes_dict):
    word_morphology = morphemes_dict[word]
    current_word = morphemes_dict['transformations']['word']
    current_word = nlp(current_word)
    transformations_roots = set(morphemes_dict['transformations']['roots'])
    transformations_prefixes = set(morphemes_dict['transformations']['prefixes'])
    transformations_suffixes = set(morphemes_dict['transformations']['suffixes'])
    # print(transformations_roots)
    if len(set(word_morphology['roots'])) > 0 and set(word_morphology['roots']).intersection(transformations_roots):
        next_word = word_morphology['word']
        next_word = nlp(next_word)
        if current_word.text in nlp.vocab and next_word.text in nlp.vocab:
            cos_similarity_list.append(current_word.similarity(next_word))
            print(current_word, next_word, current_word.similarity(next_word))
    elif len(set(word_morphology['prefixes'])) > 0 and set(word_morphology['prefixes']).intersection(transformations_prefixes):
        next_word = word_morphology['word']
        next_word = nlp(next_word)
        if current_word.text in nlp.vocab and next_word.text in nlp.vocab:
            cos_similarity_list.append(current_word.similarity(next_word))
            print(current_word, next_word, current_word.similarity(next_word))
    elif len(set(word_morphology['suffixes'])) > 0 and set(word_morphology['suffixes']).intersection(transformations_suffixes):
        next_word = word_morphology['word']
        next_word = nlp(next_word)
        if current_word.text in nlp.vocab and next_word.text in nlp.vocab:
            cos_similarity_list.append(current_word.similarity(next_word))
            print(current_word, next_word, current_word.similarity(next_word))

transformations acclimatization 0.08909641809425962
transformations acclimatizes 0.0
transformations accredits 0.03087448343277933
transformations aeroplanes 0.06991283736536134
transformations afforestation 0.14359058758303905
transformations afforests 0.0
transformations anions 0.2115951152724615
transformations antechambers 0.0
transformations antedates 0.0
transformations anterooms 0.0
transformations anticlimaxes 0.0
transformations anticyclones 0.0
transformations antiheroes 0.08067445443388084
transformations antilogarithms 0.0
transformations antimacassars 0.0
transformations antiseptics 0.06678762394989862
transformations apportionments 0.0
transformations apportions -0.004070686846721154
transformations aqualungs 0.0
transformations aquaplanes 0.0
transformations archangels 0.07853461142762282
transformations archbishoprics 0.0
transformations archbishops 0.10246728429308903
transformations archdeaconries 0.0
transformations archdeacons 0.03202775545166866
transformations arc

In [39]:
mean_cosine_similarity = np.mean(np.array(cos_similarity_list))

print(f"The mean cosine similarity of the word transformations is {mean_cosine_similarity}")

The mean cosine similarity of the word transformations is 0.07083202677578276


# Question 6A: Cosine similarity to unrelated words (3 points)

Now, do the same thing as in Question 5, but compute the mean cosine similarity between the word `"transformations"` and all words that do not share any morphemes (no shared roots, suffixes, or prefixes) with `"transformations"` at all.

In [40]:
un_cos_similarity_list = []
for i, word in enumerate(morphemes_dict):
    word_morphology = morphemes_dict[word]
    current_word = morphemes_dict['transformations']['word']
    current_word = nlp(current_word)
    transformations_roots = set(morphemes_dict['transformations']['roots'])
    transformations_prefixes = set(morphemes_dict['transformations']['prefixes'])
    transformations_suffixes = set(morphemes_dict['transformations']['suffixes'])
    # print(transformations_roots)
    if len(set(word_morphology['roots'])) > 0 and set(word_morphology['roots']).intersection(transformations_roots):
        pass
    elif len(set(word_morphology['prefixes'])) > 0 and set(word_morphology['prefixes']).intersection(transformations_prefixes):
        pass
    elif len(set(word_morphology['suffixes'])) > 0 and set(word_morphology['suffixes']).intersection(transformations_suffixes):
        pass
    else:
        next_word = word_morphology['word']
        next_word = nlp(next_word)
        if current_word.text in nlp.vocab and next_word.text in nlp.vocab:
            un_cos_similarity_list.append(current_word.similarity(next_word))
            print(current_word, next_word, current_word.similarity(next_word))

transformations acclimatize 0.04018758921627339
transformations acclimatized 0.07314213237652732
transformations acclimatizing 0.03515293125754761
transformations accreditation 0.09181227237148912
transformations accredited 0.013806972381401448
transformations accrediting 0.10674436505440278
transformations amorality 0.09986063135033896
transformations anaerobic 0.198383557699535
transformations anionic 0.15525273405111933
transformations anticlerical 0.06272549622884463
transformations anticlockwise 0.1541108546704544
transformations antislavery 0.054276793105204654
transformations apportioned 0.002797331726608952
transformations apportioning 0.017008096697752675
transformations apportionment 0.08984558321947955
transformations aquaplaning 0.030627289856364193
transformations archbishopric -0.05972396947603928
transformations archdeaconry -0.0738848878252953
transformations arising 0.2967622731716914
transformations asexual 0.23673543134998096
transformations asexuality 0.082327938304

In [42]:
un_mean_cos_similarity = np.mean(np.array(un_cos_similarity_list))

print(f"The mean cosine similarity of the word transformations to unrelated words is {un_mean_cos_similarity}")

The mean cosine similarity of the word transformations to unrelated words is 0.09066623863017562


# Question 6B: Does sharing material influence word vector similarity? (3 points)

Compare and contrast the mean similarities of the following pairs for the word `"transformations"`:

1. Shared prefixes vs. suffixes
2. Shared affixes vs. roots
3. Related (share anything) vs. Unrelated (share nothing)

Elaborate on what linguistic patterns you think lead to any differences in the above comparisons. If there are none, speculate as to why.


Shared prefixes vs. suffixes: 

There are roughly 935 words that match the suffixes for this word that have one of the suffixes "transformations" possesses. There are roughly 42 prefixes that match the word "transformations" when it comes to prefixes. As can be seen, the similarity score for unrelated words is higher than for related words, which may be owing to the higher quantity of suffix words. In most cases, shared prefixes are more useful than shared prefixes when determining word similarity.

Shared affixes vs. roots: 

The mean word similarities for roots are often larger than for affixes, which is due to the fact that affixes, which are mostly suffixes, add unneeded words, causing the word similarities to fall. Furthermore, the root of the word influences the word's direction and, in general, it aids in the improvement of the similarity score.

Related (share anything) vs. Unrelated (share nothing):

For the word "transformations," the unrelated similarity scores are 0.01 greater than the related ones. It's possible that the "s" suffix in the suffixes list contributed to the lower cosine similarity score. However, in general, connected people have a greater similarity score than unrelated people.


# Question 7A: Sharing more than one morpheme (8 points)

Compute means for 0, 1, 2, 3, etc. shared morphemes with `"transformations"`

In [50]:
# code for question 8
sharing_four_morpheme = []
sharing_three_morpheme = []
sharing_two_morpheme = []
sharing_one_morpheme = []
sharing_zero_morpheme = []
for i, word in enumerate(morphemes_dict):
    word_morphology = morphemes_dict[word]
    current_word = morphemes_dict['transformations']['word']
    current_word = nlp(current_word)
    transformations_roots = set(morphemes_dict['transformations']['roots'])
    transformations_prefixes = set(morphemes_dict['transformations']['prefixes'])
    transformations_suffixes = set(morphemes_dict['transformations']['suffixes'])
    root_count = len(set(word_morphology['roots']).intersection(transformations_roots))
    prefixes_count = len(set(word_morphology['prefixes']).intersection(transformations_prefixes))
    suffixes_count = len(set(word_morphology['suffixes']).intersection(transformations_suffixes))
    total_count = root_count + prefixes_count + suffixes_count
    if total_count == 4:
        next_word = word_morphology['word']
        next_word = nlp(next_word)
        if current_word.text in nlp.vocab and next_word.text in nlp.vocab:
            sharing_four_morpheme.append(current_word.similarity(next_word))
    if total_count == 3:
        next_word = word_morphology['word']
        next_word = nlp(next_word)
        if current_word.text in nlp.vocab and next_word.text in nlp.vocab:
            sharing_three_morpheme.append(current_word.similarity(next_word))
    if total_count == 2:
        next_word = word_morphology['word']
        next_word = nlp(next_word)
        if current_word.text in nlp.vocab and next_word.text in nlp.vocab:
            sharing_two_morpheme.append(current_word.similarity(next_word))
    if total_count == 1:
        next_word = word_morphology['word']
        next_word = nlp(next_word)
        if current_word.text in nlp.vocab and next_word.text in nlp.vocab:
            sharing_one_morpheme.append(current_word.similarity(next_word))
    if total_count == 0:
        next_word = word_morphology['word']
        next_word = nlp(next_word)
        if current_word.text in nlp.vocab and next_word.text in nlp.vocab:
            sharing_zero_morpheme.append(current_word.similarity(next_word))

mean_sharing_four_morpheme = np.mean(np.array(sharing_four_morpheme))
mean_sharing_three_morpheme = np.mean(np.array(sharing_three_morpheme))
mean_sharing_two_morpheme = np.mean(np.array(sharing_two_morpheme))
mean_sharing_one_morpheme = np.mean(np.array(sharing_one_morpheme))
mean_sharing_zero_morpheme = np.mean(np.array(sharing_zero_morpheme))

print('The mean value for 4 shared morphemes is {}'.format(mean_sharing_four_morpheme))
print('The mean value for 3 shared morphemes is {}'.format(mean_sharing_three_morpheme))
print('The mean value for 2 shared morphemes is {}'.format(mean_sharing_two_morpheme))
print('The mean value for 1 shared morphemes is {}'.format(mean_sharing_one_morpheme))
print('The mean value for 0 shared morphemes is {}'.format(mean_sharing_zero_morpheme))

The mean value for 4 shared morphemes is 1.0
The mean value for 3 shared morphemes is 0.3956008752068832
The mean value for 2 shared morphemes is 0.14856527339225958
The mean value for 1 shared morphemes is 0.06556800779531505
The mean value for 0 shared morphemes is 0.09066623863017562


# Question 7B: Free response

What about sharing more than one morpheme? How does similarity change depending on the number of shared morphemes (in any position)? Why do you think this is the case?

With the rise in common morphemes, the cosine similarity score of word changes with all related words increases. The cosine similarity score for the four shared is higher than the three shared and three shared is higher than two shared---etc, as shown in the above result. Furthermore, the four shared morphemes have a greater similarity score than the three shared morphemes. As a result, the greater the similarity score, the more shared morphemes there are, which is true for practically all words.

# Bonus: 5 points

## Bonus A: Code

Do Question 7 but for all words in `morphemes.json`.

In [ ]:
# Bonus A: Code computing mean similarity by amount of shared morphology for all words
for i in range(0, len(morphemes)):
    u_sharing_four_morpheme = []
    u_sharing_three_morpheme = []
    u_sharing_two_morpheme = []
    u_sharing_one_morpheme = []
    u_sharing_zero_morpheme = []
    for j in range(0, len(morphemes)):
        if morphemes[i] != morphemes[j]:
            current_word_morphology = morphemes[i]
            next_word_morphology = morphemes[j]
            current_word = morphemes[i]['word']
            current_word = nlp(current_word)
            next_word = morphemes[j]['word']
            next_word = nlp(next_word)
            root_count = len(set(current_word_morphology['roots']).intersection(set(next_word_morphology['roots'])))
            prefixes_count = len(set(current_word_morphology['prefixes']).intersection(set(next_word_morphology['prefixes'])))
            suffixes_count = len(set(current_word_morphology['suffixes']).intersection(set(next_word_morphology['suffixes'])))
            total_count = root_count + prefixes_count + suffixes_count
            if total_count == 4:
                if current_word.text in nlp.vocab and next_word.text in nlp.vocab:
                    u_sharing_four_morpheme.append(current_word.similarity(next_word))
            if total_count == 3:
                if current_word.text in nlp.vocab and next_word.text in nlp.vocab:
                    u_sharing_three_morpheme.append(current_word.similarity(next_word))
            if total_count == 2:
                if current_word.text in nlp.vocab and next_word.text in nlp.vocab:
                    u_sharing_two_morpheme.append(current_word.similarity(next_word))
            if total_count == 1:
                if current_word.text in nlp.vocab and next_word.text in nlp.vocab:
                    u_sharing_one_morpheme.append(current_word.similarity(next_word))
            if total_count == 0:
                if current_word.text in nlp.vocab and next_word.text in nlp.vocab:
                    u_sharing_zero_morpheme.append(current_word.similarity(next_word))

    u_mean_sharing_four_morpheme = np.mean(np.array(u_sharing_four_morpheme))
    u_mean_sharing_three_morpheme = np.mean(np.array(u_sharing_three_morpheme))
    u_mean_sharing_two_morpheme = np.mean(np.array(u_sharing_two_morpheme))
    u_mean_sharing_one_morpheme = np.mean(np.array(u_sharing_one_morpheme))
    u_mean_sharing_zero_morpheme = np.mean(np.array(u_sharing_zero_morpheme))

    print('The mean value for 4 shared morphemes for the {} is {}'.format(current_word, u_mean_sharing_four_morpheme))
    print('The mean value for 3 shared morphemes for the {} is {}'.format(current_word, u_mean_sharing_three_morpheme))
    print('The mean value for 2 shared morphemes for the {} is {}'.format(current_word, u_mean_sharing_two_morpheme))
    print('The mean value for 1 shared morphemes for the {} is {}'.format(current_word, u_mean_sharing_one_morpheme))
    print('The mean value for 0 shared morphemes for the {} is {}'.format(current_word, u_mean_sharing_zero_morpheme))

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


The mean value for 4 shared morphemes for the acclimatization is nan
The mean value for 3 shared morphemes for the acclimatization is 0.46087807822591326
The mean value for 2 shared morphemes for the acclimatization is 0.13570846086820748
The mean value for 1 shared morphemes for the acclimatization is 0.10475785171626004
The mean value for 0 shared morphemes for the acclimatization is 0.0746797279368133


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


The mean value for 4 shared morphemes for the acclimatize is nan
The mean value for 3 shared morphemes for the acclimatize is 0.5203131187422381
The mean value for 2 shared morphemes for the acclimatize is nan
The mean value for 1 shared morphemes for the acclimatize is 0.14970334154796622
The mean value for 0 shared morphemes for the acclimatize is 0.11569340913161064


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

The mean value for 4 shared morphemes for the bemoans is nan
The mean value for 3 shared morphemes for the bemoans is nan
The mean value for 2 shared morphemes for the bemoans is 0.21915372395343902
The mean value for 1 shared morphemes for the bemoans is 0.13588754681275603
The mean value for 0 shared morphemes for the bemoans is 0.10032113902672728


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


The mean value for 4 shared morphemes for the benighted is nan
The mean value for 3 shared morphemes for the benighted is nan
The mean value for 2 shared morphemes for the benighted is 0.30756709622374895
The mean value for 1 shared morphemes for the benighted is 0.20156843214277817
The mean value for 0 shared morphemes for the benighted is 0.1244583086348779
The mean value for 4 shared morphemes for the benightedly is nan
The mean value for 3 shared morphemes for the benightedly is nan
The mean value for 2 shared morphemes for the benightedly is nan
The mean value for 1 shared morphemes for the benightedly is nan
The mean value for 0 shared morphemes for the benightedly is nan
The mean value for 4 shared morphemes for the benumbed is nan
The mean value for 3 shared morphemes for the benumbed is nan
The mean value for 2 shared morphemes for the benumbed is nan
The mean value for 1 shared morphemes for the benumbed is nan
The mean value for 0 shared morphemes for the benumbed is nan


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


The mean value for 4 shared morphemes for the beribboned is nan
The mean value for 3 shared morphemes for the beribboned is nan
The mean value for 2 shared morphemes for the beribboned is 0.2942731722494078
The mean value for 1 shared morphemes for the beribboned is 0.1618948248429017
The mean value for 0 shared morphemes for the beribboned is 0.08328570311481144
The mean value for 4 shared morphemes for the beseemed is nan
The mean value for 3 shared morphemes for the beseemed is nan
The mean value for 2 shared morphemes for the beseemed is nan
The mean value for 1 shared morphemes for the beseemed is nan
The mean value for 0 shared morphemes for the beseemed is nan


## Bonus B: Free response

Describe the mean similarity patterns that you see once we look at all words in our vocabulary. Is it the same or different from the results you got in Question 7?

The above results display the mean values of each word with the rest of the words which are based on shared morphemes. As you can see that mean value is high for most shared morphemes which is similar to results in question 7. But, most of the words has only 3 shared morphemes at max and because of this, the mean value for 4 shared morphemes is coming an `nan` which means no value exists. Moreover, there is no constant values between one word to another. 

# Submission (1 point)

Please submit the `.ipynb` file and name it:

LastName_FirstName_HW3.ipynb